In [1]:
import os
import sys
sys.path.append(os.chdir('../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#from sklearn.multioutput import MultiOutputRegressor
from src.algorithms.multioutput import MultiOutputRegressor, RegressorChain

from sklearn.utils import shuffle
#from catboost import CatBoostRegressor
from src.algorithms.catboost_model import CatBoost

from sklearn.metrics import mean_absolute_error

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

import src.data as data
import src.utility as utility

In [3]:
#dataset_train = base_df.merge(data.base_dataset(), how='left')
X, Y = data.dataset(onehot=False, drop_index_columns=True)

caching base dataset train


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3291: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
print(X.shape)
print(Y.shape)
print()
print(Y.dtypes)

(33202, 75)
(33202, 4)

SPEED_AVG_Y_0    float64
SPEED_AVG_Y_1    float64
SPEED_AVG_Y_2    float64
SPEED_AVG_Y_3    float64
dtype: object


## Preprocess data

In [6]:
X.head(10)

,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-10,SPEED_AVG_-9,SPEED_AVG_-8,SPEED_AVG_-7,SPEED_AVG_-6,SPEED_AVG_-5,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-10,SPEED_SD_-9,SPEED_SD_-8,SPEED_SD_-7,SPEED_SD_-6,SPEED_SD_-5,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-10,SPEED_MAX_-9,SPEED_MAX_-8,SPEED_MAX_-7,SPEED_MAX_-6,SPEED_MAX_-5,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-10,SPEED_MIN_-9,SPEED_MIN_-8,SPEED_MIN_-7,SPEED_MIN_-6,SPEED_MIN_-5,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-10,N_VEHICLES_-9,N_VEHICLES_-8,N_VEHICLES_-7,N_VEHICLES_-6,N_VEHICLES_-5,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-10,WEATHER_-9,WEATHER_-8,WEATHER_-7,WEATHER_-6,WEATHER_-5,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-10,DISTANCE_-9,DISTANCE_-8,DISTANCE_-7,DISTANCE_-6,DISTANCE_-5,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1
0,1.0,2.0,1.0,11,Veicolo_in_avaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.6642,199.3150,204.7759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.4457,52.0643,43.0659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263.734,292.858,294.476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,129.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quasi Sereno,Quasi Sereno,Quasi Sereno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,38.0,38.0
1,1.0,2.0,1.0,6,Gestione_viabilita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.9124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.8807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352.724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2.0,1.0,6,Gestione_viabilita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.9124,201.9654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.8807,39.7029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,352.724,286.386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2.0,1.0,6,Gestione_viabilita,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,213.5957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.7280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,334.926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rovescio Nelle Vicinanze con Pioggia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0
4,1.0,2.0,1.0,2,Manutenzione_opere_in_verde,216.0995,218.6922,215.4648,214.3988,213.8432,214.4452,210.7883,213.6824,223.9108,223.1689,32.2995,29.4705,31.6018,27.5030,30.8558,32.2447,34.5211,33.1529,32.9331,30.7040,284.768,309.038,354.342,288.004,310.656,296.094,281.532,343.016,389.938,341.398,59.866,64.720,74.428,74.428,61.484,59.866,59.866,61.484,64.720,59.866,218.0,216.0,239.0,234.0,218.0,242.0,231.0,228.0,222.0,239.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,90.0,90.0,90.0,90.0,226.0,226.0,226.0,226.0,38.0,38.0
5,1.0,2.0,1.0,2,Manutenzione_opere_in_verde,214.3988,213.8432,214.4452,210.7883,213.6824,223.9108,223.1689,218.5277,219.2802,214.6672,27.5030,30.8558,32.2447,34.5211,33.1529,32.9331,30.7040,29.5927,28.9895,32.1558,288.004,310.656,296.094,281.532,343.016,389.938,341.398,328.454,300.948,286.386,74.428,61.484,59.866,59.866,61.484,64.720,59.866,59.866,61.484,59.866,234.0,218.0,242.0,231.0,228.0,222.0,239.0,265.0,255.0,258.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,90.0,226.0,226.0,226.0,226.0,38.0,38.0,38.0,38.0,38.0
6,1.0,2.0,1.0,2,Manutenzione_opere_in_verde,213.8432,214.4452,210.7883,213.6824,223.9108,223.1689,218.5277,219.2802,214.6672,216.8678,30.8558,32.2447,34.5211,33.1529,32.9331,30.7040,29.5927,28.9895,32.1558,29.6041,310.656,296.094,281.532,343.016,389.938,341.398,328.454,300.948,286.386,310.656,61.484,59.866,59.866,61.484,64.720,59.866,59.866,61.484,59.866,59.866,218.0,242.0,231.0,228.0,222

In [12]:
X.fillna(0, inplace=True)
#Y.fillna(0, inplace=True)

weather_cols = [f'WEATHER_{i}' for i in range(-10,0)]
categorical_cols = ['EMERGENCY_LANE', 'ROAD_TYPE', 'EVENT_DETAIL','EVENT_TYPE',] + weather_cols

## Train/Validation split

In [10]:
#X, Y = shuffle(X, Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [5]:
def evaluate(X_test, y_test):
    mask_test = np.all(y_test.notnull(), axis=1)

    y_pred = model.predict(X_test[mask_test])
    return mean_absolute_error(y_test[mask_test], y_pred)

## Fit

In [13]:
catboost = CatBoost(params_dict={
    'X': X,
    'loss_function': 'MAE',
    'eval_metric': 'MAE',
    'iterations':2,
    'depth':6,
    'learning_rate':1,
    'cat_features': categorical_cols,
    'verbose': False
})

In [14]:
model = MultiOutputRegressor(catboost, n_jobs=2)
model.fit(X, Y)

ValueError: Found input variables with inconsistent numbers of samples: [33202, 34083]

In [8]:
model = RegressorChain(catboost)
model.fit(X, Y)

catboost: fitting...
0:	learn: 179.4956168	test: 175.3456680	best: 175.3456680 (0)	total: 212ms	remaining: 6m 20s
1:	learn: 178.9968195	test: 174.8467500	best: 174.8467500 (1)	total: 340ms	remaining: 5m 5s
2:	learn: 178.4984960	test: 174.3491379	best: 174.3491379 (2)	total: 476ms	remaining: 4m 45s
3:	learn: 178.0010185	test: 173.8549984	best: 173.8549984 (3)	total: 610ms	remaining: 4m 33s
4:	learn: 177.5028299	test: 173.3578717	best: 173.3578717 (4)	total: 752ms	remaining: 4m 29s
5:	learn: 177.0041521	test: 172.8590717	best: 172.8590717 (5)	total: 893ms	remaining: 4m 26s
6:	learn: 176.5060759	test: 172.3632696	best: 172.3632696 (6)	total: 1.06s	remaining: 4m 31s
7:	learn: 176.0079174	test: 171.8650074	best: 171.8650074 (7)	total: 1.23s	remaining: 4m 36s
8:	learn: 175.5096610	test: 171.3671520	best: 171.3671520 (8)	total: 1.39s	remaining: 4m 36s
9:	learn: 175.0112370	test: 170.8685286	best: 170.8685286 (9)	total: 1.52s	remaining: 4m 32s
10:	learn: 174.5124542	test: 170.3692944	best: 170

RegressorChain(base_estimator=<src.algorithms.catboost_model.CatBoost object at 0x119732208>,
        cv=None, order=None, random_state=None)

In [11]:
evaluate(X_test, y_test)

7.8075137571574285

## Baseline

Catboost: 7.822071069208421

Catboost chain: 7.8075137571574285